In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Testing The Bowyer Watson Algorithm for triangulation

In [3]:
%load_ext Cython

In [25]:
%%cython
cimport cython
from libc.stdlib cimport malloc, free
from cython cimport sizeof, NULL
from libc.math cimport sqrt






###########################


###########################
"""First layer simplex. The Vertex"""
"""Below functions to create an instance of the struct"""
ctypedef struct Vertex:
    double x
    double y
    int vid_nr

    


cdef Vertex create_vertex(double x_coord, double y_coord, int vid_number):
    cdef Vertex v
    v.x = x_coord
    v.y = y_coord
    v.vid_nr = vid_number
    return v





##
"""The Face"""
"""Functions to create instances of Face"""
ctypedef struct Face:
    Vertex ver1
    Vertex ver2
    Vertex ver3
    int fid_nr


    

cdef Face create_face(Vertex vertex1, Vertex vertex2, Vertex vertex3, int fid_number):
    cdef Face f
    f.ver1 = vertex1
    f.ver2 = vertex2
    f.ver3 = vertex3
    f.fid_nr = fid_number
    return f







###########################


###########################
"""Below functions for dynamic allocation purposes"""
###########################


###########################



"""deallocates and moves all elements under the removed index 1 upward"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* dealloc_from_varray(Vertex* original_array, int current_size, int remove_index):
    #careful remove_index is actual position - 1

    cdef Vertex* new_array = <Vertex*>malloc((current_size-1) * sizeof(Vertex))
    cdef int i
    cdef int j
    cdef int upper_limit = remove_index
    for i in range(upper_limit):
        new_array[i] = original_array[i]
    for j in range(upper_limit+1, current_size):
        new_array[j-1] = original_array[j]
        
    free(original_array)

    
    return new_array
        


"""A function like the one below to realloc a vertex array instead of a face array"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Vertex* realloc_to_varray(Vertex* old_array, int current_size, Vertex new_vertex):
    cdef int new_size = current_size + 1
    cdef Vertex* new_array = <Vertex*>malloc(new_size * sizeof(Vertex))
    cdef int i
    for i in range(current_size):
        new_array[i] = old_array[i]
    new_array[current_size] = new_vertex
    
    free(old_array)

    return new_array



"""A function that takes a pointer to an array of Face structs. It creates a new array 1 size larger."""
"""It then appends the -new face- to it. Returns the new array. Frees the old one from memory"""
"""ATTENTION!!::: THE FACE* MESH INPUT MUST BE A MALLOC'ED ARRAY OR A POINTER TO A STATICALLY DECLARED ONE BY REFERENCING"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Face* realloc_to_mesh(Face* mesh, int current_size, Face new_face):
    cdef int new_size = current_size + 1
    cdef Face* new_mesh = <Face*>malloc(new_size * sizeof(Face))
    cdef int i
    for i in range(current_size):
        new_mesh[i] = mesh[i]
    new_mesh[current_size] = new_face
    free(mesh)

    return new_mesh


"""Removes a Face in a mesh of a certain index"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* dealloc_from_mesh(Face* original_mesh, int current_size, int remove_index):

    cdef Face* new_mesh = <Face*>malloc((current_size-1) * sizeof(Face))
    cdef int i
    cdef int j
    cdef int upper_limit = remove_index
    for i in range(upper_limit):
        new_mesh[i] = original_mesh[i]
    for j in range(upper_limit+1, current_size):
        new_mesh[j-1] = original_mesh[j]
    free(original_mesh)

    
    return new_mesh




""""""###
"""A function to jumptstart allocate the Vertex struct in an array for testing purposes"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* alloc_varr(int size, double var):
    cdef Vertex* varray = <Vertex*>malloc(size * sizeof(Vertex))
    cdef int i
    for i in range(size):
        varray[i] = create_vertex(var, var, i)
    return varray

############################

"""Below functions for useful geometrical comparisons"""

############################




"""return magnitude of vector"""
#works
cdef inline double magnitude(double x, double y):
    return sqrt(x*x+y*y)



"""return dot product with respect to origin"""
#works
cdef inline double dot_product(Vertex v1, Vertex v2):
    return v1.x * v2.x + v1.y * v2.y



"""Distance between vertices"""
#works

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double distance(Vertex v1, Vertex v2):
    return sqrt((v1.x - v2.x)**2 + (v1.y - v2.y)**2)



"""Circumcenter"""
#works

#input 3 vertexes to calculate circumcenter of circumcircle
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline Vertex circumcenter(Vertex v1, Vertex v2, Vertex v3):
    cdef Vertex center
    #inverse of determinant method (by wedge product)
    cdef double D = (v1.x*(v2.y-v3.y) + v2.x*(v3.y-v1.y) + v3.x*(v1.y-v2.y))*2
    center.x = (1/D)*((v1.x**2 + v1.y**2)*(v2.y-v3.y) + (v2.x**2 + v2.y**2)*(v3.y-v1.y) + (v3.x**2 + v3.y**2)*(v1.y-v2.y))
    center.y = (1/D)*((v1.x**2 + v1.y**2)*(v3.x-v2.x) + (v2.x**2 + v2.y**2)*(v1.x-v3.x) + (v3.x**2 + v3.y**2)*(v2.x-v1.x))
    center.vid_nr = -1
    return center


"""Circumradius"""
#works

#you have to be retarded to not know what this returns
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double circumradius(Vertex v1, Vertex v2, Vertex v3):
    cdef Vertex ccenter = circumcenter(v1, v2, v3)
    cdef double radius = sqrt((v1.x-ccenter.x)**2 + (v1.y-ccenter.y)**2)
    
    return radius



"""Check if Vertex p is in Circumcircle of v1v2v3"""
#works

#returns 0 or 1 based on condition if p is in C(v)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline int in_circumcircle(Face f, Vertex p):
    cdef Vertex c_center = circumcenter(f.ver1, f.ver2, f.ver3)
    cdef double c_radius = circumradius(f.ver1, f.ver2, f.ver3)
    cdef double distance_from_circumcenter = distance(c_center, p)
    return distance_from_circumcenter < c_radius




"""Create the super Triangle"""
"""First create the max rectangle"""
"""Then set the triangle with sqrt3 formulas"""
#works

#return a pointer to a [3] array with 3 Vertex objects
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face create_super_triangle(Vertex* points, int num_points):
    cdef Face super_triangle
    
    cdef double min_x = points[0].x
    cdef double min_y = points[0].y
    cdef double max_x = points[3].x
    cdef double max_y = points[3].y

    cdef int i

    for i in range(num_points):
        if points[i].x < min_x:
            min_x = points[i].x

        if points[i].y < min_y:
            min_y = points[i].y
            
    for i in range(num_points):
        if points[i].x > max_x:
            max_x = points[i].x

        if points[i].y > max_y:
            max_y = points[i].y
            
    cdef double a = (max_x - min_x)
    cdef double b = (max_y - min_y)

    

    super_triangle.ver1.x = min_x + a*0.5
    super_triangle.ver1.y = max_y + a*0.5
    super_triangle.ver1.vid_nr = -1


    super_triangle.ver2.x = max_x + b
    super_triangle.ver2.y = min_y - 1
    super_triangle.ver2.vid_nr = -2


    super_triangle.ver3.x = min_x - b
    super_triangle.ver3.y = min_y - 1
    super_triangle.ver3.vid_nr = -3

    
    super_triangle.fid_nr = 0


    return super_triangle










"""The most important part to have valid faces with no crossings. It takes a list of free vertexes and oriens them c-clockwise"""
"""Uses the Dot product orthogonality and Bubble Sort"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void sort_by_angle_cclockwise(Vertex* free_vertexes, Vertex center_vertex, int size):
    cdef int vertexes_over_x_axis = 0
    cdef int vertexes_under_x_axis = 0
    ###  !DON'T FORGET TO FREE!  ###
    ###
    cdef Vertex* displaced_vertexes_overx = <Vertex*>malloc(sizeof(Vertex))
    cdef Vertex* displaced_vertexes_underx = <Vertex*>malloc(sizeof(Vertex))
    ###
    

    cdef double x_newcoord
    cdef double y_newcoord
    
    cdef int i
    
    for i in range(size):
        x_newcoord = free_vertexes[i].x - center_vertex.x
        y_newcoord = free_vertexes[i].y - center_vertex.y

        if y_newcoord >= 0.:
            displaced_vertexes_overx = realloc_to_varray(displaced_vertexes_overx, vertexes_over_x_axis, free_vertexes[i])
            vertexes_over_x_axis = vertexes_over_x_axis + 1
            
            
        elif y_newcoord < 0.:
            displaced_vertexes_underx = realloc_to_varray(displaced_vertexes_underx, vertexes_under_x_axis, free_vertexes[i])
            vertexes_under_x_axis = vertexes_under_x_axis + 1
        

    cdef int j
    cdef int k
    cdef int l
    cdef int p
    cdef Vertex tmp_vertex1
    cdef Vertex tmp_vertex2
    cdef double tmp_mag1
    cdef double tmp_mag2

    cdef double tmp_projection1
    cdef double tmp_projection2


    for j in range(vertexes_over_x_axis):
        for k in range(vertexes_over_x_axis-j-1):
            tmp_mag1 = magnitude((displaced_vertexes_overx[k].x-center_vertex.x), (displaced_vertexes_overx[k].y-center_vertex.y))
            tmp_mag2 = magnitude((displaced_vertexes_overx[k+1].x-center_vertex.x), (displaced_vertexes_overx[k+1].y-center_vertex.y))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            
            tmp_projection1 = (displaced_vertexes_overx[k].x-center_vertex.x)/tmp_mag1
            tmp_projection2 = (displaced_vertexes_overx[k+1].x-center_vertex.x)/tmp_mag2

            if tmp_projection1 <= tmp_projection2:
                tmp_vertex1 = displaced_vertexes_overx[k]
                displaced_vertexes_overx[k] = displaced_vertexes_overx[k+1]
                displaced_vertexes_overx[k+1] = tmp_vertex1
                
    for l in range(vertexes_under_x_axis):
        for p in range(vertexes_under_x_axis-l-1):
            tmp_mag1 = (magnitude(displaced_vertexes_underx[p].x-center_vertex.x, displaced_vertexes_underx[p].y-center_vertex.y))
            tmp_mag2 = (magnitude(displaced_vertexes_underx[p+1].x-center_vertex.x, displaced_vertexes_underx[p+1].y-center_vertex.y))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            
            tmp_projection1 = (displaced_vertexes_underx[p].x-center_vertex.x)/tmp_mag1
            tmp_projection2 = (displaced_vertexes_underx[p+1].x-center_vertex.x)/tmp_mag2


            if tmp_projection1 > tmp_projection2:
                tmp_vertex2 = displaced_vertexes_underx[p]
                displaced_vertexes_underx[p] = displaced_vertexes_underx[p+1]
                displaced_vertexes_underx[p+1] = tmp_vertex2
 
    cdef int m
    cdef int n
    for m in range(vertexes_over_x_axis):
        free_vertexes[m] = displaced_vertexes_overx[m]
    

    for n in range(vertexes_over_x_axis, size):
        free_vertexes[n] = displaced_vertexes_underx[n-vertexes_over_x_axis]

 
    free(displaced_vertexes_overx)
    free(displaced_vertexes_underx)
#end function




    
    
    
    

#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* triangulate(Vertex* original_array, int varr_size, int* gm_size):
    
    cdef Face* final_mesh = <Face*>malloc(sizeof(Face))
    cdef int final_mesh_size = 0
    
    cdef Face* bad_mesh = <Face*>malloc(sizeof(Face))
    cdef Vertex* free_vertices = <Vertex*>malloc(sizeof(Vertex))
    cdef int bad_mesh_size = 0
    cdef int free_vertices_size = 0

    cdef Face temp_face
    cdef Face super_triangle = create_super_triangle(original_array, varr_size)
    
    final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, super_triangle)
    final_mesh_size = final_mesh_size + 1


    cdef int q, r, i, k, l, m, s
    for r in range(varr_size): #main loop

        q = 0
        
        while True:
            if q == final_mesh_size:
                break
            if in_circumcircle(final_mesh[q], original_array[r]):
                bad_mesh = realloc_to_mesh(bad_mesh, bad_mesh_size, final_mesh[q])
                bad_mesh_size = bad_mesh_size + 1
                final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, q)
                final_mesh_size = final_mesh_size - 1
            else:
                q = q + 1

        for i in range(bad_mesh_size):
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[i].ver1)
            free_vertices_size = free_vertices_size + 1
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[i].ver2)
            free_vertices_size = free_vertices_size + 1
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[i].ver3)
            free_vertices_size = free_vertices_size + 1

        for n in range(bad_mesh_size):
            bad_mesh = dealloc_from_mesh(bad_mesh, bad_mesh_size, 0)
        bad_mesh_size = 0

        k=0
        while k < free_vertices_size - 1:
            l = k + 1
            while l < free_vertices_size:
                if free_vertices[l].vid_nr == free_vertices[k].vid_nr:
                    free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, l)
                    free_vertices_size = free_vertices_size - 1
                else:
                    l = l + 1
            k = k + 1

        sort_by_angle_cclockwise(free_vertices, original_array[r], free_vertices_size)

        for m in range(free_vertices_size):
            temp_face = create_face(original_array[r], free_vertices[m], free_vertices[(m+1)%free_vertices_size], 0)
            final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, temp_face)
            final_mesh_size = final_mesh_size + 1
            
        for s in range(free_vertices_size):
            free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, 0)
        free_vertices_size = 0

    cdef int index = 0

    while True:
        if index == final_mesh_size:
            break
        
        if final_mesh[index].ver1.vid_nr < 0 or final_mesh[index].ver2.vid_nr < 0 or final_mesh[index].ver3.vid_nr < 0:
            final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, index)
            final_mesh_size = final_mesh_size - 1

        else:
            index = index + 1

    cdef int z
    
    for z in range(final_mesh_size):
        final_mesh[z].fid_nr = z
        
    gm_size[0] = final_mesh_size
        
    return final_mesh
        



    

"""unit test for angular sorting and using the dealloc functions on a mesh and varray"""
#start the array of Zero-Zero Vertices
#!! THIS ARRAY IS MALLOCED BUT NOT FREED BY DEFAULT !! THE SIZE HAS TO BE KEPT TRACK OF
cdef int varr_ssize = 4
cdef int size2 = 8
cdef Vertex* og_array = alloc_varr(varr_ssize, 0)
cdef Vertex* spinarr = alloc_varr(size2, 0)
cdef int* global_mesh_size = <int*>malloc(sizeof(int))
global_mesh_size[0] = 0 #dynamically changed through triangulate fx

og_array[0].x = 0.2
og_array[0].y = 2
og_array[1].x = -1.8
og_array[1].y = 0.15
og_array[2].x = -0.1
og_array[2].y = -2.3
og_array[3].x = 2
og_array[3].y = -0.3
#og_array[4].x = -1.8
#og_array[4].y = 6
#og_array[5].x = -3.9
#og_array[5].y = -2
#og_array[6].x = 1.7
#og_array[6].y = -3.4
#og_array[6].vid_nr = og_array[5].vid_nr
#og_array[2].vid_nr = og_array[4].vid_nr
#og_array[3].vid_nr = og_array[4].vid_nr

spinarr[0].x = 0.2
spinarr[0].y = 2
spinarr[1].x = -1.8
spinarr[1].y = 0.15
spinarr[2].x = -0.1
spinarr[2].y = -2.3
spinarr[3].x = 4
spinarr[3].y = -0.3
spinarr[4].x = 0.1
spinarr[4].y = 3.9
#spinarr[4].vid_nr = -1
spinarr[5].x = -1
spinarr[5].y = 1
#spinarr[5].vid_nr = 4
spinarr[6].x = -3.1
spinarr[6].y = -2.3
#spinarr[6].vid_nr = -3
spinarr[7].x = 2.7
spinarr[7].y = -3.3
#spinarr[7].vid_nr = -2
cdef int compare = 2
cdef Vertex* doublecompare = alloc_varr(compare, 0)
doublecompare[0].x = 0.2
doublecompare[0].y = 2
doublecompare[0].vid_nr = 0
doublecompare[1].x = 0.1
doublecompare[1].y = 3.9
doublecompare[1].vid_nr = -1

cdef Vertex og = create_vertex(2, -0.1, 0)
print(varr_ssize)
for i in range(varr_ssize):
    print(og_array[i])

print(size2, "spinarr size")    
print(size2)
for i in range(size2):
    print(spinarr[i])
    
sort_by_angle_cclockwise(spinarr, og, size2)

print("\nsorted\n-----")
for i in range(size2):
    print(spinarr[i])


cdef Face* testmesh = triangulate(og_array, varr_ssize, global_mesh_size)

print("\nglobal mesh size is:", global_mesh_size[0])
for f in range(global_mesh_size[0]):
    print(testmesh[f])

free(testmesh)
free(og_array)
free(spinarr)
free(doublecompare)
free(global_mesh_size)












4
{'x': 0.2, 'y': 2.0, 'vid_nr': 0}
{'x': -1.8, 'y': 0.15, 'vid_nr': 1}
{'x': -0.1, 'y': -2.3, 'vid_nr': 2}
{'x': 2.0, 'y': -0.3, 'vid_nr': 3}
8 spinarr size
8
{'x': 0.2, 'y': 2.0, 'vid_nr': 0}
{'x': -1.8, 'y': 0.15, 'vid_nr': 1}
{'x': -0.1, 'y': -2.3, 'vid_nr': 2}
{'x': 4.0, 'y': -0.3, 'vid_nr': 3}
{'x': 0.1, 'y': 3.9, 'vid_nr': 4}
{'x': -1.0, 'y': 1.0, 'vid_nr': 5}
{'x': -3.1, 'y': -2.3, 'vid_nr': 6}
{'x': 2.7, 'y': -3.3, 'vid_nr': 7}

sorted
-----
{'x': 0.1, 'y': 3.9, 'vid_nr': 4}
{'x': 0.2, 'y': 2.0, 'vid_nr': 0}
{'x': -1.0, 'y': 1.0, 'vid_nr': 5}
{'x': -1.8, 'y': 0.15, 'vid_nr': 1}
{'x': -3.1, 'y': -2.3, 'vid_nr': 6}
{'x': -0.1, 'y': -2.3, 'vid_nr': 2}
{'x': 2.7, 'y': -3.3, 'vid_nr': 7}
{'x': 4.0, 'y': -0.3, 'vid_nr': 3}

global mesh size is: 2
{'ver1': {'x': 2.0, 'y': -0.3, 'vid_nr': 3}, 'ver2': {'x': 0.2, 'y': 2.0, 'vid_nr': 0}, 'ver3': {'x': -1.8, 'y': 0.15, 'vid_nr': 1}, 'fid_nr': 0}
{'ver1': {'x': 2.0, 'y': -0.3, 'vid_nr': 3}, 'ver2': {'x': -1.8, 'y': 0.15, 'vid_nr': 1}, 'ver

In [ ]:
%%cython
cimport cython
from libc.stdlib cimport malloc, free
from cython cimport sizeof, NULL
from libc.math cimport sqrt






###########################


###########################
"""First layer simplex. The Vertex"""
"""Below functions to create an instance of the struct"""
ctypedef struct Vertex:
    double x
    double y
    int vid_nr

    


cdef Vertex create_vertex(double x_coord, double y_coord, int vid_number):
    cdef Vertex v
    v.x = x_coord
    v.y = y_coord
    v.vid_nr = vid_number
    return v





##
"""The Face"""
"""Functions to create instances of Face"""
ctypedef struct Face:
    Vertex ver1
    Vertex ver2
    Vertex ver3
    int fid_nr


    

cdef Face create_face(Vertex vertex1, Vertex vertex2, Vertex vertex3, int fid_number):
    cdef Face f
    f.ver1 = vertex1
    f.ver2 = vertex2
    f.ver3 = vertex3
    f.fid_nr = fid_number
    return f







###########################


###########################
"""Below functions for dynamic allocation purposes"""
###########################


###########################



"""deallocates and moves all elements under the removed index 1 upward"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* dealloc_from_varray(Vertex* original_array, int current_size, int remove_index):
    #careful remove_index is actual position - 1

    cdef Vertex* new_array = <Vertex*>malloc((current_size-1) * sizeof(Vertex))
    cdef int i
    cdef int j
    cdef int upper_limit = remove_index
    for i in range(upper_limit):
        new_array[i] = original_array[i]
    for j in range(upper_limit+1, current_size):
        new_array[j-1] = original_array[j]
        
    free(original_array)

    
    return new_array
        


"""A function like the one below to realloc a vertex array instead of a face array"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Vertex* realloc_to_varray(Vertex* old_array, int current_size, Vertex new_vertex):
    cdef int new_size = current_size + 1
    cdef Vertex* new_array = <Vertex*>malloc(new_size * sizeof(Vertex))
    cdef int i
    for i in range(current_size):
        new_array[i] = old_array[i]
    new_array[current_size] = new_vertex
    
    free(old_array)

    return new_array



"""A function that takes a pointer to an array of Face structs. It creates a new array 1 size larger."""
"""It then appends the -new face- to it. Returns the new array. Frees the old one from memory"""
"""ATTENTION!!::: THE FACE* MESH INPUT MUST BE A MALLOC'ED ARRAY OR A POINTER TO A STATICALLY DECLARED ONE BY REFERENCING"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Face* realloc_to_mesh(Face* mesh, int current_size, Face new_face):
    cdef int new_size = current_size + 1
    cdef Face* new_mesh = <Face*>malloc(new_size * sizeof(Face))
    cdef int i
    for i in range(current_size):
        new_mesh[i] = mesh[i]
    new_mesh[current_size] = new_face
    free(mesh)

    return new_mesh


"""Removes a Face in a mesh of a certain index"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* dealloc_from_mesh(Face* original_mesh, int current_size, int remove_index):

    cdef Face* new_mesh = <Face*>malloc((current_size-1) * sizeof(Face))
    cdef int i
    cdef int j
    cdef int upper_limit = remove_index
    for i in range(upper_limit):
        new_mesh[i] = original_mesh[i]
    for j in range(upper_limit+1, current_size):
        new_mesh[j-1] = original_mesh[j]
    free(original_mesh)

    
    return new_mesh




""""""###
"""A function to jumptstart allocate the Vertex struct in an array for testing purposes"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* alloc_varr(int size, double var):
    cdef Vertex* varray = <Vertex*>malloc(size * sizeof(Vertex))
    cdef int i
    for i in range(size):
        varray[i] = create_vertex(var, var, i)
    return varray

############################

"""Below functions for useful geometrical comparisons"""

############################




"""return magnitude of vector"""
#works
cdef inline double magnitude(double x, double y):
    return sqrt(x*x+y*y)



"""return dot product with respect to origin"""
#works
cdef inline double dot_product(Vertex v1, Vertex v2):
    return v1.x * v2.x + v1.y * v2.y



"""Distance between vertices"""
#works

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double distance(Vertex v1, Vertex v2):
    return sqrt((v1.x - v2.x)**2 + (v1.y - v2.y)**2)



"""Circumcenter"""
#works

#input 3 vertexes to calculate circumcenter of circumcircle
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline Vertex circumcenter(Vertex v1, Vertex v2, Vertex v3):
    cdef Vertex center
    #inverse of determinant method (by wedge product)
    cdef double D = (v1.x*(v2.y-v3.y) + v2.x*(v3.y-v1.y) + v3.x*(v1.y-v2.y))*2
    center.x = (1/D)*((v1.x**2 + v1.y**2)*(v2.y-v3.y) + (v2.x**2 + v2.y**2)*(v3.y-v1.y) + (v3.x**2 + v3.y**2)*(v1.y-v2.y))
    center.y = (1/D)*((v1.x**2 + v1.y**2)*(v3.x-v2.x) + (v2.x**2 + v2.y**2)*(v1.x-v3.x) + (v3.x**2 + v3.y**2)*(v2.x-v1.x))
    center.vid_nr = -1
    return center


"""Circumradius"""
#works

#you have to be retarded to not know what this returns
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double circumradius(Vertex v1, Vertex v2, Vertex v3):
    cdef Vertex ccenter = circumcenter(v1, v2, v3)
    cdef double radius = sqrt((v1.x-ccenter.x)**2 + (v1.y-ccenter.y)**2)
    
    return radius



"""Check if Vertex p is in Circumcircle of v1v2v3"""
#works

#returns 0 or 1 based on condition if p is in C(v)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline int in_circumcircle(Face f, Vertex p):
    cdef Vertex c_center = circumcenter(f.ver1, f.ver2, f.ver3)
    cdef double c_radius = circumradius(f.ver1, f.ver2, f.ver3)
    cdef double distance_from_circumcenter = distance(c_center, p)
    return distance_from_circumcenter < c_radius




"""Create the super Triangle"""
"""First create the max rectangle"""
"""Then set the triangle with sqrt3 formulas"""
#works

#return a pointer to a [3] array with 3 Vertex objects
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face create_super_triangle(Vertex* points, int num_points):
    cdef Face super_triangle
    
    cdef double min_x = points[0].x
    cdef double min_y = points[0].y
    cdef double max_x = points[3].x
    cdef double max_y = points[3].y

    cdef int i

    for i in range(num_points):
        if points[i].x < min_x:
            min_x = points[i].x

        if points[i].y < min_y:
            min_y = points[i].y
            
    for i in range(num_points):
        if points[i].x > max_x:
            max_x = points[i].x

        if points[i].y > max_y:
            max_y = points[i].y
            
    cdef double a = (max_x - min_x)
    cdef double b = (max_y - min_y)

    

    super_triangle.ver1.x = min_x + a*0.5
    super_triangle.ver1.y = max_y + a*0.5
    super_triangle.ver1.vid_nr = -1


    super_triangle.ver2.x = max_x + b
    super_triangle.ver2.y = min_y - 1
    super_triangle.ver2.vid_nr = -2


    super_triangle.ver3.x = min_x - b
    super_triangle.ver3.y = min_y - 1
    super_triangle.ver3.vid_nr = -3

    
    super_triangle.fid_nr = 0


    return super_triangle










"""The most important part to have valid faces with no crossings. It takes a list of free vertexes and oriens them c-clockwise"""
"""Uses the Dot product orthogonality and Bubble Sort"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void sort_by_angle_cclockwise(Vertex* free_vertexes, Vertex center_vertex, int size):
    cdef int vertexes_over_x_axis = 0
    cdef int vertexes_under_x_axis = 0
    ###  !DON'T FORGET TO FREE!  ###
    ###
    cdef Vertex* displaced_vertexes_overx = <Vertex*>malloc(sizeof(Vertex))
    cdef Vertex* displaced_vertexes_underx = <Vertex*>malloc(sizeof(Vertex))
    ###
    

    cdef double x_newcoord
    cdef double y_newcoord
    
    cdef int i
    
    for i in range(size):
        x_newcoord = free_vertexes[i].x - center_vertex.x
        y_newcoord = free_vertexes[i].y - center_vertex.y

        if y_newcoord >= 0.:
            displaced_vertexes_overx = realloc_to_varray(displaced_vertexes_overx, vertexes_over_x_axis, free_vertexes[i])
            vertexes_over_x_axis = vertexes_over_x_axis + 1
            
            
        elif y_newcoord < 0.:
            displaced_vertexes_underx = realloc_to_varray(displaced_vertexes_underx, vertexes_under_x_axis, free_vertexes[i])
            vertexes_under_x_axis = vertexes_under_x_axis + 1
        
    #for f in range(vertexes_over_x_axis):
        #print("\n", displaced_vertexes_overx[f])
    #print("---")
    #for f in range(vertexes_under_x_axis):
        #print("\n", displaced_vertexes_underx[f])
    cdef int j
    cdef int k
    cdef int l
    cdef int p
    cdef Vertex tmp_vertex1
    cdef Vertex tmp_vertex2
    cdef double tmp_mag1
    cdef double tmp_mag2

    cdef double tmp_projection1
    cdef double tmp_projection2


    #print("\niteration over x\n")
    for j in range(vertexes_over_x_axis):
        for k in range(vertexes_over_x_axis-j-1):
            tmp_mag1 = magnitude((displaced_vertexes_overx[k].x-center_vertex.x), (displaced_vertexes_overx[k].y-center_vertex.y))
            tmp_mag2 = magnitude((displaced_vertexes_overx[k+1].x-center_vertex.x), (displaced_vertexes_overx[k+1].y-center_vertex.y))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            
            tmp_projection1 = (displaced_vertexes_overx[k].x-center_vertex.x)/tmp_mag1
            tmp_projection2 = (displaced_vertexes_overx[k+1].x-center_vertex.x)/tmp_mag2
            #print("\ntmp projection 1vs2\n:", "vid:", displaced_vertexes_overx[k].vid_nr, ":" , tmp_projection1 , "vid:", displaced_vertexes_overx[k+1].vid_nr, ":" , tmp_projection2)

            if tmp_projection1 <= tmp_projection2:
                #print(tmp_projection1 <= tmp_projection2)
                tmp_vertex1 = displaced_vertexes_overx[k]
                displaced_vertexes_overx[k] = displaced_vertexes_overx[k+1]
                displaced_vertexes_overx[k+1] = tmp_vertex1
                
    
    #print("\niteration under x\n")
    for l in range(vertexes_under_x_axis):
        for p in range(vertexes_under_x_axis-l-1):
            tmp_mag1 = (magnitude(displaced_vertexes_underx[p].x-center_vertex.x, displaced_vertexes_underx[p].y-center_vertex.y))
            tmp_mag2 = (magnitude(displaced_vertexes_underx[p+1].x-center_vertex.x, displaced_vertexes_underx[p+1].y-center_vertex.y))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            
            tmp_projection1 = (displaced_vertexes_underx[p].x-center_vertex.x)/tmp_mag1
            tmp_projection2 = (displaced_vertexes_underx[p+1].x-center_vertex.x)/tmp_mag2
            #print("\ntmp projection 1vs2\n:", "vid:", displaced_vertexes_underx[p].vid_nr, ":" , tmp_projection1 , "vid:", displaced_vertexes_underx[p+1].vid_nr, ":" , tmp_projection2)


            if tmp_projection1 > tmp_projection2:
                #print(tmp_projection1 <= tmp_projection2)
                tmp_vertex2 = displaced_vertexes_underx[p]
                displaced_vertexes_underx[p] = displaced_vertexes_underx[p+1]
                displaced_vertexes_underx[p+1] = tmp_vertex2
 
    cdef int m
    cdef int n
    for m in range(vertexes_over_x_axis):
        free_vertexes[m] = displaced_vertexes_overx[m]
    

    for n in range(vertexes_over_x_axis, size):
        free_vertexes[n] = displaced_vertexes_underx[n-vertexes_over_x_axis]
        
    #for f in range(vertexes_over_x_axis):
        #print("\n", displaced_vertexes_overx[f])
    #print("---")
    #for f in range(vertexes_under_x_axis):
        #print("\n", displaced_vertexes_underx[f])
    
    #for f in range(size):
        #print(free_vertexes[f])
 
    free(displaced_vertexes_overx)
    free(displaced_vertexes_underx)
#end function




    
    
    
    

#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* triangulate(Vertex* original_array, int varr_size, int* gm_size):
    
    cdef Face* final_mesh = <Face*>malloc(sizeof(Face))
    cdef int final_mesh_size = 0
    
    cdef Face* bad_mesh = <Face*>malloc(sizeof(Face))
    cdef Vertex* free_vertices = <Vertex*>malloc(sizeof(Vertex))
    cdef int bad_mesh_size = 0
    cdef int free_vertices_size = 0

    cdef Face temp_face
    cdef Face super_triangle = create_super_triangle(original_array, varr_size)
    
    final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, super_triangle)
    final_mesh_size = final_mesh_size + 1
    
    #print("super triangle appended to final_mesh")
    #print("\nsize is now:", final_mesh_size,"\n")
    #print(final_mesh[0])
    
    #print("\nstarting main loop with og arr size:", varr_size)

    cdef int q, r, i, k, l, m, s
    for r in range(varr_size): #main loop
        #print("\n-----------\nat 'r' main iteration:", r)

        q = 0
        #print("\nq was set to =", q,"\n----\n entering While True loop over final mesh size\n--\n")
        
        while True:
            #print("\nin while loop at iteration q=:", q)
            if q == final_mesh_size:
                #print("\nbreaking at q=", q,"\nlast checked q index was q-1 =", q-1)
                break
            if in_circumcircle(final_mesh[q], original_array[r]): #if-statement tested. works
                #print("\nvid_nr:", original_array[r].vid_nr, "is in circumcircle of :", final_mesh[q],"\nproceeding to throw the face in bad_mesh arr")
                bad_mesh = realloc_to_mesh(bad_mesh, bad_mesh_size, final_mesh[q])
                #print("\nbad_mesh was allocated the face above. Proceeding to increase size by 1")
                bad_mesh_size = bad_mesh_size + 1
                #print("\nsize successfully increased bad mesh size by 1 to : ", bad_mesh_size,"\nproceeding to dealloc index:",q," from final_mesh")
                final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, q)
                #print("\nsuccessfully dealloced, proceeding to decrease size")
                final_mesh_size = final_mesh_size - 1
                #print("\nsize decreased to:", final_mesh_size)
            else:
                #print("\n----->>>>entered else condition, increasing q by 1")
                q = q + 1
                #print("\nq increased by 1 to:", q)
        #print("\nbed mesh size is now:", bad_mesh_size)

        for i in range(bad_mesh_size):
            #print("\nentered i loop over bad_mesh. Allocing each vertex of bed_mesh elements to free_vertices")
            #print("\ncurrently at bad element:\n", bad_mesh[i])
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[i].ver1)
            free_vertices_size = free_vertices_size + 1
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[i].ver2)
            free_vertices_size = free_vertices_size + 1
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[i].ver3)
            free_vertices_size = free_vertices_size + 1
            
            #print("\nalloced ver1,2,3 to free array. current size is:", free_vertices_size)
        #print("\n--> now entering important n loop to empty bad_mesh. will try to print bad_mesh[0] and SHOULD get out of bound element")    
        for n in range(bad_mesh_size):
            bad_mesh = dealloc_from_mesh(bad_mesh, bad_mesh_size, 0)
        bad_mesh_size = 0
        #print("bad_mesh[0]:\n", bad_mesh[0])
        #print("\nhopefully freed bad mesh of all elements and size ready for next iteration")
        #print("\nwill now print the whole free_vertices array to check who is in it")
        #for g in range(free_vertices_size):
            #print("\n", free_vertices[g])
        #remove duplicates from free_vertices
        #print("\nremoving duplicates now:")
        k=0
        while k < free_vertices_size - 1:
            l = k + 1
            while l < free_vertices_size:
                if free_vertices[l].vid_nr == free_vertices[k].vid_nr:
                    free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, l)
                    free_vertices_size = free_vertices_size - 1
                else:
                    l = l + 1
            k = k + 1
        #print("\nremoving duplicates:")
        #for g in range(free_vertices_size):
            #print(free_vertices[g])    
        #sort by angle
        sort_by_angle_cclockwise(free_vertices, original_array[r], free_vertices_size)
        #print("\nsort by angle now with reference to point:", r)
        #for g in range(free_vertices_size):
            #print(free_vertices[g])
        #print("\nentering m loop over free_vertices.\nhere we append the faces created by the r'th vertex with free_vertices\n")
        for m in range(free_vertices_size):
            temp_face = create_face(original_array[r], free_vertices[m], free_vertices[(m+1)%free_vertices_size], 0)
            #print("created temp face:\n", temp_face,"\nallocing to final_mesh")
            final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, temp_face)
            #print("\nsuccessfully allocated to final_mesh. increasing size by 1")
            final_mesh_size = final_mesh_size + 1
            #print("\nfinal mesh size increased by 1 to:", final_mesh_size)
            
            
        #print("\n--> entering important s loop to complete zero free_vertices in preparation for the next r'th iteration")    
        for s in range(free_vertices_size):
            free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, 0)
        free_vertices_size = 0
        #print("\nexpected out of bounds free_vertices[0]:", free_vertices[0])
        
        #print("\n-entering a loop to see what is inside final_mesh\n")
        #for g in range(final_mesh_size):
            #print("\n", final_mesh[g])


    
    #print("\n--entering the loop to remove the super triangle")
    #remove super triag
    cdef int index = 0
    #print("\nentering main True while loop\n")
    #print("current final_mesh_size is: ", final_mesh_size)
    while True:
        #print("\n", index)
        if index == final_mesh_size:
            #print("\nbreaking at index = ", index)
            break
        
        if final_mesh[index].ver1.vid_nr < 0 or final_mesh[index].ver2.vid_nr < 0 or final_mesh[index].ver3.vid_nr < 0:
            #print("\nfound element to dealloc that might be of the super triangle:\n", final_mesh[index])
            final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, index)
            final_mesh_size = final_mesh_size - 1
            #print("\n final mesh size changed to:", final_mesh_size)
        else:
            index = index + 1
        
        #print("\nindex incremented +1 at=", index)
        
    #change fid's in order
    cdef int z
    
    for z in range(final_mesh_size):
        #print("\nreseting all mesh fids to ordered numbers")
        final_mesh[z].fid_nr = z
        #print("\n", final_mesh[z])
        
    gm_size[0] = final_mesh_size
        
    return final_mesh
        



    

"""unit test for angular sorting and using the dealloc functions on a mesh and varray"""
#start the array of Zero-Zero Vertices
#!! THIS ARRAY IS MALLOCED BUT NOT FREED BY DEFAULT !! THE SIZE HAS TO BE KEPT TRACK OF
cdef int varr_ssize = 4
cdef int size2 = 8
cdef Vertex* og_array = alloc_varr(varr_ssize, 0)
cdef Vertex* spinarr = alloc_varr(size2, 0)
cdef int* global_mesh_size = <int*>malloc(sizeof(int))
global_mesh_size[0] = 0 #dynamically changed through triangulate fx

og_array[0].x = 0.2
og_array[0].y = 2
og_array[1].x = -1.8
og_array[1].y = 0.15
og_array[2].x = -0.1
og_array[2].y = -2.3
og_array[3].x = 2
og_array[3].y = -0.3
#og_array[4].x = -1.8
#og_array[4].y = 6
#og_array[5].x = -3.9
#og_array[5].y = -2
#og_array[6].x = 1.7
#og_array[6].y = -3.4
#og_array[6].vid_nr = og_array[5].vid_nr
#og_array[2].vid_nr = og_array[4].vid_nr
#og_array[3].vid_nr = og_array[4].vid_nr

spinarr[0].x = 0.2
spinarr[0].y = 2
spinarr[1].x = -1.8
spinarr[1].y = 0.15
spinarr[2].x = -0.1
spinarr[2].y = -2.3
spinarr[3].x = 4
spinarr[3].y = -0.3
spinarr[4].x = 0.1
spinarr[4].y = 3.9
#spinarr[4].vid_nr = -1
spinarr[5].x = -1
spinarr[5].y = 1
#spinarr[5].vid_nr = 4
spinarr[6].x = -3.1
spinarr[6].y = -2.3
#spinarr[6].vid_nr = -3
spinarr[7].x = 2.7
spinarr[7].y = -3.3
#spinarr[7].vid_nr = -2
cdef int compare = 2
cdef Vertex* doublecompare = alloc_varr(compare, 0)
doublecompare[0].x = 0.2
doublecompare[0].y = 2
doublecompare[0].vid_nr = 0
doublecompare[1].x = 0.1
doublecompare[1].y = 3.9
doublecompare[1].vid_nr = -1

cdef Vertex og = create_vertex(2, -0.1, 0)
print(varr_ssize)
for i in range(varr_ssize):
    print(og_array[i])

print(size2, "spinarr size")    
print(size2)
for i in range(size2):
    print(spinarr[i])
    
sort_by_angle_cclockwise(spinarr, og, size2)

print("\nsorted\n-----")
for i in range(size2):
    print(spinarr[i])


cdef Face* testmesh = triangulate(og_array, varr_ssize, global_mesh_size)

print("\nglobal mesh size is:", global_mesh_size[0])
for f in range(global_mesh_size[0]):
    print(testmesh[f])

free(testmesh)
free(og_array)
free(spinarr)
free(doublecompare)
free(global_mesh_size)












In [ ]:
k = 10

for i in range(5):
    while True:
        print(k, i)
        if i==k:
            break
        i = i+1

In [ ]:
k = 10
i = 0
for i in range(k):
    i = i  + 1
    print(k, i)
    k = k - 1
    
print(k)

In [ ]:
k = 10

for i in range(k):
    print(k, "k and i", i)
    k = k-1

In [ ]:
#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* triangulate(Vertex* original_array, int varr_size) nogil:
    
    cdef Face* final_mesh = <Face*>malloc(sizeof(Face))
    cdef Face* bad_mesh = <Face*>malloc(sizeof(Face))
    cdef Vertex* free_vertices = <Vertex*>malloc(sizeof(Vertex))
    
    cdef int final_mesh_size = 0
    cdef int bad_mesh_size = 0
    cdef int free_vertices_size = 0

    cdef Face super_triangle = create_super_triangle(original_array, varr_size)
    
    final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, super_triangle)
    final_mesh_size = final_mesh_size + 1

    
    cdef int i
    cdef int j
    cdef int k
    cdef int m
    cdef int n
    cdef int l
    cdef int r
    
    cdef Face tmp_face
    

    for i in range(varr_size):

        for j in range(final_mesh_size):

            if in_circumcircle(final_mesh[j], original_array[i]): #if-statement tested. works
                bad_mesh = realloc_to_mesh(bad_mesh, bad_mesh_size, final_mesh[j])
                bad_mesh_size = bad_mesh_size + 1
                final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, j)
                final_mesh_size = final_mesh_size - 1
        

    
        for m in range(bad_mesh_size):
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver1)
            free_vertices_size = free_vertices_size + 1
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver2)
            free_vertices_size = free_vertices_size + 1
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver3)
            free_vertices_size = free_vertices_size + 1
            
        for r in range(bad_mesh_size):
            bad_mesh = dealloc_from_mesh(bad_mesh, bad_mesh_size, 0)
            bad_mesh_size = bad_mesh_size - 1
        

        for k in range(free_vertices_size-1):
            for l in range(k+1, free_vertices_size):
                if free_vertices[l].vid_nr == free_vertices[k].vid_nr:
                    free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, l)
                    free_vertices_size = free_vertices_size - 1
                    

        sort_by_angle_cclockwise(free_vertices, original_array[i], free_vertices_size) #works post-loop. tested
        

        
        tmp_face = create_face(original_array[i], free_vertices[free_vertices_size-1], free_vertices[0], 0)
        final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
        final_mesh_size = final_mesh_size + 1
        for n in range(free_vertices_size - 1):
            tmp_face = create_face(original_array[i], free_vertices[n], free_vertices[n+1], 0)
            final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
            final_mesh_size = final_mesh_size + 1
    free(bad_mesh)
    free(free_vertices)
    
    cdef int p
    cdef int q
    
    for p in range(final_mesh_size):
        if final_mesh[p].ver1.vid_nr < 0 or final_mesh[p].ver2.vid_nr < 0 or final_mesh[p].ver3.vid_nr < 0:
            final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, p)
            final_mesh_size = final_mesh_size - 1
    
    for q in range(final_mesh_size):
        final_mesh[q].fid_nr = q
    
    return final_mesh

In [ ]:
cdef Face* final_mesh
cdef Face* bad_mesh 
cdef Vertex* free_vertices
    
cdef int final_mesh_size = 0
cdef int bad_mesh_size = 0
cdef int free_vertices_size = 0

cdef Face super_triangle = create_super_triangle(original_array, varr_size)
  
final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, super_triangle)
final_mesh_size += 1

    
cdef int i
cdef int j
cdef int k
cdef int m
cdef int n
cdef int l
cdef int r
   
cdef Face tmp_face
    

for i in range(varr_size):
    print("iterating at ",i,"'th vertex")
    for j in range(final_mesh_size):
        print("iterating at ",j,"'th triangle in final mesh")

        if in_circumcircle(final_mesh[j], original_array[i]):
            print(i,"'th vertex is in circle of ",j,"'th triangle in final messh")
            bad_mesh = realloc_to_mesh(bad_mesh, bad_mesh_size, final_mesh[j])
            print("realloced to bad mesh\n")
            bad_mesh_size = bad_mesh_size + 1
            print("bad mesh size:", bad_mesh_size)
            final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, j)
            print('\ndealloced from main mesh\n')
            final_mesh_size = final_mesh_size - 1
            print("final mesh size:", final_mesh_size)
        

        
    for m in range(bad_mesh_size):
            
        free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver1)
        free_vertices_size = free_vertices_size + 1
            
        free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver2)
        free_vertices_size = free_vertices_size + 1
            
        free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver3)
        free_vertices_size = free_vertices_size + 1
        
        print("free vertecess size:", free_vertices_size)
        
        for s in range(free_vertices_size):
            print(free_vertices[s].vid_nr)
            
        #bad_mesh = dealloc_from_mesh(bad_mesh, bad_mesh_size, m)
        #bad_mesh_size = bad_mesh_size - 1
        #for r in range(bad_mesh_size):
        #bad_mesh = dealloc_from_mesh(bad_mesh, bad_mesh_size, r-r)
        #bad_mesh_size = bad_mesh_size - 1
        

    for k in range(free_vertices_size-1):
        for l in range(k+1, free_vertices_size):
            if free_vertices[l].vid_nr == free_vertices[k].vid_nr:
                print("deallocing vid_nr:", free_vertices[l].vid_nr)
                
                free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, l)
                
                free_vertices_size = free_vertices_size - 1
                
                print("size:", free_vertices_size)
                    

    sort_by_angle_clockwise(free_vertices, original_array[i], free_vertices_size)
        

        
    tmp_face = create_face(original_array[i], free_vertices[free_vertices_size-1], free_vertices[0], 0)
    final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
    final_mesh_size = final_mesh_size + 1
    for n in range(free_vertices_size - 1):
        tmp_face = create_face(original_array[i], free_vertices[n], free_vertices[n+1], 0)
        final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
        final_mesh_size = final_mesh_size + 1
free(bad_mesh)
free(free_vertices)
    
cdef int p
cdef int q
    
for p in range(final_mesh_size):
    if final_mesh[p].ver1.vid_nr < 0 or final_mesh[p].ver2.vid_nr < 0 or final_mesh[p].ver3.vid_nr < 0:
        final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, p)
        final_mesh_size = final_mesh_size - 1
    
for q in range(final_mesh_size):
    final_mesh[q].fid_nr = q
    print(final_mesh[q])



free(original_array)
free(final_mesh)



In [ ]:
##test unit for mesh plot

cdef Face f1 = create_face(vertex_array[0], vertex_array[1], vertex_array[2], 0)
cdef Face f2 = create_face(vertex_array[1], vertex_array[2], vertex_array[3], 1)
cdef Face f3 = create_face(vertex_array[1], vertex_array[3], vertex_array[6], 2)


cdef int mesh_size = 0

mesh = realloc_to_mesh(mesh, mesh_size, f1)
mesh_size = mesh_size + 1
mesh = realloc_to_mesh(mesh, mesh_size, f2)
mesh_size = mesh_size + 1
mesh = realloc_to_mesh(mesh, mesh_size, f3)
mesh_size = mesh_size + 1


print("\nmesh size is:", mesh_size,"\n")


print(mesh[0],"\n")
print(mesh[1],"\n")
print(mesh[2],"\n")



# Extract x and y coordinates for each vertex of both triangles
  # Add the first vertex at the end to close the triangle
x_coords_triangle1 = [mesh[0].ver1.x, mesh[0].ver2.x, mesh[0].ver3.x, mesh[0].ver1.x]
y_coords_triangle1 = [mesh[0].ver1.y, mesh[0].ver2.y, mesh[0].ver3.y, mesh[0].ver1.y] 


x_coords_triangle2 = [mesh[1].ver1.x, mesh[1].ver2.x, mesh[1].ver3.x, mesh[1].ver1.x]
y_coords_triangle2 = [mesh[1].ver1.y, mesh[1].ver2.y, mesh[1].ver3.y, mesh[1].ver1.y] 


x_coords_triangle3 = [mesh[2].ver1.x, mesh[2].ver2.x, mesh[2].ver3.x, mesh[2].ver1.x]
y_coords_triangle3 = [mesh[2].ver1.y, mesh[2].ver2.y, mesh[2].ver3.y, mesh[2].ver1.y] 


# Plot both triangles
fig, ax = plt.subplots()
triangle1 = patches.Polygon(xy=list(zip(x_coords_triangle1, y_coords_triangle1)), closed=True, facecolor='lightblue', edgecolor='black')
triangle2 = patches.Polygon(xy=list(zip(x_coords_triangle2, y_coords_triangle2)), closed=True, facecolor='lightgreen', edgecolor='black')
triangle3 = patches.Polygon(xy=list(zip(x_coords_triangle3, y_coords_triangle3)), closed=True, facecolor='red', edgecolor='black')
ax.add_patch(triangle1)
ax.add_patch(triangle2)
ax.add_patch(triangle3)

# Calculate the center of each triangle
center_x_triangle1 = sum(x_coords_triangle1) / 3.0
center_y_triangle1 = sum(y_coords_triangle1) / 3.0

center_x_triangle2 = sum(x_coords_triangle2) / 3.0
center_y_triangle2 = sum(y_coords_triangle2) / 3.0

center_x_triangle3 = sum(x_coords_triangle3) / 3.0
center_y_triangle3 = sum(y_coords_triangle3) / 3.0

# Add labels "1" and "2" at the centers of the triangles
ax.text(center_x_triangle1, center_y_triangle1, "f1", fontsize=12, ha='center', va='center')
ax.text(center_x_triangle2, center_y_triangle2, "f2", fontsize=12, ha='center', va='center')
ax.text(center_x_triangle3, center_y_triangle3, "f3", fontsize=12, ha='center', va='center')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Three Triangles')
ax.grid(False)
plt.axis('equal')  # Set aspect ratio to be equal
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define the vertices of the first triangle (x, y coordinates)
vertex1 = (0, 0)
vertex2 = (3, 4)
vertex3 = (6, 0)

# Define the vertices of the second triangle (x, y coordinates)
vertex4 = (7, 3)

# Extract x and y coordinates for each vertex of both triangles
x_coords_triangle1 = [vertex1[0], vertex2[0], vertex3[0], vertex1[0]]  # Add the first vertex at the end to close the triangle
y_coords_triangle1 = [vertex1[1], vertex2[1], vertex3[1], vertex1[1]]

x_coords_triangle2 = [vertex2[0], vertex3[0], vertex4[0], vertex2[0]]  # Add the first vertex at the end to close the triangle
y_coords_triangle2 = [vertex2[1], vertex3[1], vertex4[1], vertex2[1]]

# Plot both triangles
fig, ax = plt.subplots()
triangle1 = patches.Polygon(xy=list(zip(x_coords_triangle1, y_coords_triangle1)), closed=True, facecolor='lightblue', edgecolor='blue')
triangle2 = patches.Polygon(xy=list(zip(x_coords_triangle2, y_coords_triangle2)), closed=True, facecolor='lightgreen', edgecolor='green')
ax.add_patch(triangle1)
ax.add_patch(triangle2)

# Calculate the center of each triangle
center_x_triangle1 = sum(x_coords_triangle1) / 3.0
center_y_triangle1 = sum(y_coords_triangle1) / 3.0

center_x_triangle2 = sum(x_coords_triangle2) / 3.0
center_y_triangle2 = sum(y_coords_triangle2) / 3.0

# Add labels "1" and "2" at the centers of the triangles
ax.text(center_x_triangle1, center_y_triangle1, "1", fontsize=12, ha='center', va='center')
ax.text(center_x_triangle2, center_y_triangle2, "2", fontsize=12, ha='center', va='center')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Two Triangles')
ax.grid(True)
plt.axis('equal')  # Set aspect ratio to be equal
plt.show()


In [ ]:
%%cython

"""the main triangulation function"""
#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* triangulate_vertices(Vertex* original_array, int varr_size) nogil:
    
    cdef Face* final_mesh = <Face*>malloc(sizeof(Face))
    cdef Face* bad_mesh
    cdef Vertex* free_vertices
    
    cdef int final_mesh_size = 1
    cdef int bad_mesh_size = 0
    cdef int free_vertices_size = 0

    cdef Face super_triangle = create_super_triangle(original_array, varr_size)
    
    final_mesh[0] = super_triangle

    
    cdef size_t i
    cdef int j
    cdef size_t k
    cdef size_t m
    cdef size_t n
    cdef int l
    
    cdef Face tmp_face
    
    #main loop over all vertexes present in the og array
    for i in range(varr_size):
        #for each vertex loop over every Face in the final_mesh
        for j in range(final_mesh_size):
            #if the current vertex in the array is in the Face we are at, add the face to a bad_mesh array
            #increase size of bad mesh array by 1
            #deallocate the face from the final_mesh array
            #decrease the size of final mesh by 1
            if in_circumcircle(final_mesh[j], original_array[i]):
                bad_mesh = realloc_to_mesh(bad_mesh, bad_mesh_size, final_mesh[j])
                bad_mesh_size = bad_mesh_size + 1
                final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, j)
                final_mesh_size = final_mesh_size - 1
        
        #the second for iteration goes through the bad mesh and allocates all triangles vertixes to a free_array
        
        for m in range(bad_mesh_size):
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver1)
            free_vertices_size = free_vertices_size + 1
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver2)
            free_vertices_size = free_vertices_size + 1
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver3)
            free_vertices_size = free_vertices_size + 1
        
        #in this loop the repeated vertixes are removed the remaining will be prepared for sorting by angle
        #the size of the array is kept track of everytime a vertex is remeaved due to it being repeated
        for k in range(free_vertices_size-1):
            for l in range(k+1, free_vertices_size):
                if free_vertices[l].vid_nr == free_vertices[k].vid_nr:
                    free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, l)
                    free_vertices_size = free_vertices_size - 1
                    
        #now we sort the free_vertices by angle clockwise
        sort_by_angle_clockwise(free_vertices, original_array[i], free_vertices_size)
        
        #now we create a face in clockwise pairs for every 2-nearest couple in free_vertex + the origin, original)array[i]
        #then we allocate this face to final_mesh and update size before the main 'for' -i loop repeats.
        #first allocate the last Face to avoid 
        
        tmp_face = create_face(original_array[i], free_vertices[free_vertices_size-1], free_vertices[0], 0)
        final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
        final_mesh_size = final_mesh_size + 1
        for n in range(free_vertices_size-1):
            tmp_face = create_face(original_array[i], free_vertices[n], free_vertices[n+1], 0)
            final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
            final_mesh_size = final_mesh_size + 1
    free(bad_mesh)
    free(free_vertices)
    
    cdef int p
    cdef int q
    
    for p in range(final_mesh_size):
        if final_mesh[p].ver1.vid_nr < 0 or final_mesh[p].ver2.vid_nr < 0 or final_mesh[p].ver3.vid_nr < 0:
            final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, p)
            final_mesh_size = final_mesh_size - 1
    
    for q in range(final_mesh_size):
        final_mesh[q].fid_nr = q
    
    return final_mesh

In [ ]:
"""unit test for angular sorting and using the dealloc functions on a mesh and varray"""
#start the array of Zero-Zero Vertices
#!! THIS ARRAY IS MALLOCED BUT NOT FREED BY DEFAULT !! THE SIZE HAS TO BE KEPT TRACK OF
cdef int varr_size = 7
cdef Vertex* vertex_array = alloc_varr(varr_size, 0)

vertex_array[0].x = 0.3
vertex_array[0].y = 2
vertex_array[1].x = 1.8
vertex_array[1].y = 0.1
vertex_array[2].x = -0.7
vertex_array[2].y = -0.1
vertex_array[3].x = 0.2
vertex_array[3].y = -2
vertex_array[4].x = -1.8
vertex_array[4].y = 6
vertex_array[5].x = -3.9
vertex_array[5].y = -2
vertex_array[6].x = 1.7
vertex_array[6].y = -3.4
cdef Vertex origin = create_vertex(0, 0, 20)


print("\nbegin by printing our array that holds all the Vertex datastructs")

for i in range(varr_size):
    print(vertex_array[i])


cdef Vertex temp_ver
cdef double mag

print("\nnow use the sort_by_angle_clockwise() function and alloc faces to mesh")


#test unit for bubble sort by angle on vertex array
sort_by_angle_clockwise(vertex_array, origin, varr_size)





cdef Face* test_mesh = <Face*>malloc(varr_size * sizeof(Face))
#insert the last permutation to avoid % algebra
test_mesh[varr_size-1] = create_face(origin, vertex_array[varr_size-1], vertex_array[0], varr_size-1)
for i in range(varr_size-1):
    test_mesh[i] = create_face(origin, vertex_array[i], vertex_array[i+1], i)
    

print("\nprinting the mesh by elements:\n")
for i in range(varr_size):
    print("face: ",i,"\n", test_mesh[i])


    
print("\nnow we test the mesh dealloc function. We want to remove by index and hope fid_nr is indexed always correctly\nWe try to remove index 5 again\n")

test_mesh = dealloc_from_mesh(test_mesh, varr_size, 5)

print("printing the mesh by elements:\n")
for i in range(varr_size-1):
    print("face: ",i,"\n", test_mesh[i])

    
print("\nthe dealloc functions works successfully")
free(vertex_array)
free(test_mesh)


In [ ]:
##general algorithm


mesh = []
available_vertexes = []
mesh.append(super_triangle)

for vertex in vertex_list:
    for face in mesh:
        if vertex in circumcircle(face, vertex):
            face.fid_nr = -1
            face.v1 -> available_vertexes
            face.v2 -> available_vertexes
            face.v3 -> available_vertexes
    create_face(vertex, all elements in available_vertex in order) -> mesh
for face in mesh:
    if face.fid_nr = -1:
        delete face

In [ ]:
cdef Face* bad_mesh = <Face*>malloc(sizeof(Face))

cdef int current_mesh_size = 0

cdef Face Face1 = create_face(vertex_array[0], vertex_array[1], vertex_array[2], 0)

cdef Face Face2 = create_face(vertex_array[2], vertex_array[3], vertex_array[4], 1)



bad_mesh = realloc_to_mesh(bad_mesh, current_mesh_size, Face1)
current_mesh_size += 1

bad_mesh = realloc_to_mesh(bad_mesh, current_mesh_size, Face2)
current_mesh_size += 1

print("\nstart with size = 0. Now i allocate 2 Faces\n")

print(bad_mesh[0],"\n")
print(bad_mesh[1],"\n")

print("current_size = ", current_mesh_size,"\nnow i deallocate a face\n")

bad_mesh = dealloc_from_mesh(bad_mesh, current_mesh_size, 0)
current_mesh_size -= 1
bad_mesh = dealloc_from_mesh(bad_mesh, current_mesh_size, 0)
current_mesh_size -= 1

print(current_mesh_size)




In [ ]:
"""unit test for angular sorting and using the dealloc functions on a mesh and varray"""
#start the array of Zero-Zero Vertices
#!! THIS ARRAY IS MALLOCED BUT NOT FREED BY DEFAULT !! THE SIZE HAS TO BE KEPT TRACK OF
cdef int varr_size = 7
cdef Vertex* vertex_array = alloc_varr(varr_size, 0)

vertex_array[0].x = 0.3
vertex_array[0].y = 2
vertex_array[1].x = 1.8
vertex_array[1].y = 0.1
vertex_array[2].x = -0.7
vertex_array[2].y = -0.1
vertex_array[3].x = 0.2
vertex_array[3].y = -2
vertex_array[4].x = -1.8
vertex_array[4].y = 6
vertex_array[5].x = -3.9
vertex_array[5].y = -2
vertex_array[6].x = 1.7
vertex_array[6].y = -3.4
cdef Vertex origin = create_vertex(0, 0, 20)


cdef Face* mesh = triangulate_vertices(vertex_array, varr_size)






free(vertex_array)
free(mesh)

In [ ]:
%%cython
cimport cython
from libc.stdlib cimport malloc, free
from libc.stdio cimport printf
from cython cimport sizeof, NULL
import numpy as np
cimport numpy as np
from libc.math cimport sqrt


ctypedef np.float64_t dtype_t

    
##
"""First layer simplex. The Vertex"""
"""Below functions to create an instance of the struct"""
ctypedef struct Vertex:
    double x
    double y
    int vid_nr

    


cdef Vertex create_vertex(double x_coord, double y_coord, int vid_number) nogil:
    cdef Vertex v
    v.x = x_coord
    v.y = y_coord
    v.vid_nr = vid_number
    return v





##
"""Third layer simples. The Face"""
"""Functions to create instances of Face"""
ctypedef struct Face:
    Vertex ver1
    Vertex ver2
    Vertex ver3
    int fid_nr


    

cdef Face create_face(Vertex vertex1, Vertex vertex2, Vertex vertex3, int fid_number) nogil:
    cdef Face f
    f.ver1 = vertex1
    f.ver2 = vertex2
    f.ver3 = vertex3
    f.fid_nr = fid_number
    return f


##
"""The second layer simplex. The edge"""
"""Function to create an Instance of Edge struct"""
ctypedef struct Edge:
    Vertex v1
    Vertex v2
    int eid_nr


cdef Edge create_edge(Vertex vertex1, Vertex vertex2, int eid_number) nogil:
    cdef Edge e
    e.v1 = vertex1
    e.v2 = vertex2
    e.eid_nr = eid_number
    return e





###########################


###########################
"""Below functions for dynamic allocation purposes"""
###########################


###########################



"""same as above. deallocates and moves all elements under the removed index 1 upward"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* dealloc_from_varray(Vertex* original_array, int current_size, int remove_index) nogil:
    #careful remove_index is actual position - 1

    cdef Vertex* new_arr = <Vertex*>malloc((current_size-1) * sizeof(Vertex))
    cdef size_t i
    cdef size_t j
    cdef int upper_limit = remove_index
    for i in range(upper_limit):
        new_arr[i] = original_array[i]
    for j in range(upper_limit+1, current_size):
        new_arr[j-1] = original_array[j]
    free(original_array)

    
    return new_arr
        


"""A function like the one below to realloc a vertex array instead of a face array"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Vertex* realloc_to_varray(Vertex* old_array, int current_size, Vertex new_vertex) nogil:
    cdef int new_size = current_size + 1
    cdef Vertex* new_array = <Vertex*>malloc(new_size * sizeof(Vertex))
    cdef size_t i
    for i in range(current_size):
        new_array[i] = old_array[i]
    new_array[current_size] = new_vertex
    free(old_array)

    return new_array



"""A function that takes a pointer to an array of Face structs. It creates a new array 1 size larger."""
"""It then appends the -new face- to it. Returns the new array. Frees the old one from memory"""
"""ATTENTION!!::: THE FACE* MESH INPUT MUST BE A MALLOC'ED ARRAY OR A POINTER TO A STATICALLY DECLARED ONE BY REFERENCING"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Face* realloc_to_mesh(Face* mesh, int current_size, Face new_face) nogil:
    cdef int new_size = current_size + 1
    cdef Face* new_mesh = <Face*>malloc(new_size * sizeof(Face))
    cdef size_t i
    for i in range(current_size):
        new_mesh[i] = mesh[i]
    new_mesh[current_size] = new_face
    free(mesh)

    return new_mesh


"""Removes a Face in a mesh of a certain index"""
#testing
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* dealloc_from_mesh(Face* original_mesh, int current_size, int remove_index) nogil:

    cdef Face* new_mesh = <Face*>malloc((current_size-1) * sizeof(Face))
    cdef size_t i
    cdef size_t j
    cdef int upper_limit = remove_index
    for i in range(upper_limit):
        new_mesh[i] = original_mesh[i]
    for j in range(upper_limit+1, current_size):
        new_mesh[j-1] = original_mesh[j]
    free(original_mesh)

    
    return new_mesh




""""""###
"""A function to jumptstart allocate the Vertex struct in an array for testing purposes"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* alloc_varr(int size, double var) nogil:
    cdef Vertex* varray = <Vertex*>malloc(size * sizeof(Vertex))
    cdef size_t i
    for i in range(size):
        varray[i] = create_vertex(var, var, i)
    return varray

############################

"""Below functions for useful geometrical comparisons"""

############################




"""return magnitude of vector"""
#works
cdef inline double magnitude(double x, double y) nogil:
    return sqrt(x*x+y*y)



"""return dot product with respect to origin"""
#works
cdef inline double dot_product(Vertex v1, Vertex v2) nogil:
    return v1.x * v2.x + v1.y * v2.y



"""Distance between vertices"""
#works

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double distance(Vertex v1, Vertex v2) nogil:
    return sqrt((v1.x - v2.x)**2 + (v1.y - v2.y)**2)



"""Circumcenter"""
#works

#input 3 vertexes to calculate circumcenter of circumcircle
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline Vertex circumcenter(Vertex v1, Vertex v2, Vertex v3) nogil:
    cdef Vertex center
    #inverse of determinant method (by wedge product)
    cdef double D = (v1.x*(v2.y-v3.y) + v2.x*(v3.y-v1.y) + v3.x*(v1.y-v2.y))*2
    center.x = (1/D)*((v1.x**2 + v1.y**2)*(v2.y-v3.y) + (v2.x**2 + v2.y**2)*(v3.y-v1.y) + (v3.x**2 + v3.y**2)*(v1.y-v2.y))
    center.y = (1/D)*((v1.x**2 + v1.y**2)*(v3.x-v2.x) + (v2.x**2 + v2.y**2)*(v1.x-v3.x) + (v3.x**2 + v3.y**2)*(v2.x-v1.x))
    
    return center


"""Circumradius"""
#works

#you have to be retarded to not know what this returns
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double circumradius(Vertex v1, Vertex v2, Vertex v3) nogil:
    cdef Vertex ccenter = circumcenter(v1, v2, v3)
    cdef double radius = sqrt((v1.x-ccenter.x)**2 + (v1.y-ccenter.y)**2)
    
    return radius



"""Check if Vertex p is in Circumcircle of v1v2v3"""
#works

#returns 0 or 1 based on condition if p is in C(v)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline int in_circumcircle(Face f, Vertex p) nogil:
    cdef Vertex c_center = circumcenter(f.ver1, f.ver2, f.ver3)
    cdef double c_radius = circumradius(f.ver1, f.ver2, f.ver3)
    cdef double distance_from_circumcenter = distance(c_center, p)
    return distance_from_circumcenter <= c_radius




"""Create the super Triangle"""
"""First create the max rectangle"""
"""Then set the triangle with sqrt3 formulas"""
#works

#return a pointer to a [3] array with 3 Vertex objects
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face create_super_triangle(Vertex* points, int num_points) nogil:
    cdef Face super_triangle
    
    cdef double min_x = points[0].x
    cdef double min_y = points[0].y
    cdef double max_x = points[3].x
    cdef double max_y = points[3].y

    cdef int i

    for i in range(num_points):
        if points[i].x < min_x:
            min_x = points[i].x

        if points[i].y < min_y:
            min_y = points[i].y
            
    for i in range(num_points):
        if points[i].x > max_x:
            max_x = points[i].x

        if points[i].y > max_y:
            max_y = points[i].y
            
    cdef double a = (max_x - min_x)
    cdef double b = (max_y - min_y)

    

    super_triangle.ver1.x = min_x + a*0.5
    super_triangle.ver1.y = max_y + a*0.5
    super_triangle.ver1.vid_nr = -1


    super_triangle.ver2.x = max_x + b
    super_triangle.ver2.y = min_y - 1
    super_triangle.ver2.vid_nr = -2


    super_triangle.ver3.x = min_x - b
    super_triangle.ver3.y = min_y - 1
    super_triangle.ver3.vid_nr = -3

    
    super_triangle.fid_nr = 0


    return super_triangle









"""The most important part to have valid faces with no crossings. It takes a list of free vertexes and oriens them c-clockwise"""
"""Uses the Dot product orthogonality and Bubble Sort"""
#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void sort_by_angle_clockwise(Vertex* free_vertexes, Vertex center_vertex, int size) nogil:
    cdef int vertexes_over_x_axis = 0
    cdef int vertexes_under_x_axis = 0
    """DON'T FORGET TO FREE"""
    ###
    cdef Vertex* displaced_vertexes_overx = <Vertex*>malloc(sizeof(Vertex))
    cdef Vertex* displaced_vertexes_underx = <Vertex*>malloc(sizeof(Vertex))
    ###
    

    cdef double x_newcoord
    cdef double y_newcoord
    
    cdef size_t i
    
    for i in range(size):
        x_newcoord = free_vertexes[i].x - center_vertex.x
        y_newcoord = free_vertexes[i].y - center_vertex.y

        if y_newcoord >= 0:
            vertexes_over_x_axis = vertexes_over_x_axis + 1
            displaced_vertexes_overx = realloc_to_varray(displaced_vertexes_overx, vertexes_over_x_axis-1, free_vertexes[i])
            
        elif y_newcoord < 0:
            vertexes_under_x_axis = vertexes_under_x_axis + 1
            displaced_vertexes_underx = realloc_to_varray(displaced_vertexes_underx, vertexes_under_x_axis-1, free_vertexes[i])
            

    cdef size_t j
    cdef size_t k
    cdef size_t l
    cdef size_t p
    cdef double tmp_mag1
    cdef double tmp_mag2
    cdef double tmp_dot_product1
    cdef double tmp_dot_product2
    cdef Vertex tmp_placeholder1
    cdef Vertex tmp_placeholder2
    cdef Vertex tmp_vertex1
    cdef Vertex x_unit = create_vertex(1, 0, 0)


    
    for j in range(vertexes_over_x_axis):
        for k in range(vertexes_over_x_axis-j-1):
            tmp_mag1 = (magnitude(displaced_vertexes_overx[k].x-center_vertex.x, displaced_vertexes_overx[k].y-center_vertex.y))
            tmp_mag2 = (magnitude(displaced_vertexes_overx[k+1].x-center_vertex.x, displaced_vertexes_overx[k+1].y-center_vertex.x))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            tmp_placeholder1 = displaced_vertexes_overx[k]
            tmp_placeholder1.x = (displaced_vertexes_overx[k].x-center_vertex.x)/tmp_mag1
            tmp_placeholder1.y = (displaced_vertexes_overx[k].y-center_vertex.y)/tmp_mag1
            
            tmp_placeholder2 = displaced_vertexes_overx[k+1]
            tmp_placeholder2.x = (displaced_vertexes_overx[k+1].x-center_vertex.x)/tmp_mag2
            tmp_placeholder2.y = (displaced_vertexes_overx[k+1].y-center_vertex.y)/tmp_mag2
            
            tmp_dot_product1 = dot_product(x_unit, tmp_placeholder1)
            tmp_dot_product2 = dot_product(x_unit, tmp_placeholder2)
            if tmp_dot_product1 > tmp_dot_product2:
                tmp_vertex1 = displaced_vertexes_overx[k]
                displaced_vertexes_overx[k] = displaced_vertexes_overx[k+1]
                displaced_vertexes_overx[k+1] = tmp_vertex1

                
                
    for l in range(vertexes_under_x_axis):
        for p in range(vertexes_under_x_axis-l-1):
            tmp_mag1 = (magnitude(displaced_vertexes_underx[p].x, displaced_vertexes_underx[p].y))
            tmp_mag2 = (magnitude(displaced_vertexes_underx[p+1].x, displaced_vertexes_underx[p+1].y))

            
            tmp_placeholder1 = displaced_vertexes_underx[p]
            tmp_placeholder1.x = displaced_vertexes_underx[p].x/tmp_mag1
            tmp_placeholder1.y = displaced_vertexes_underx[p].y/tmp_mag1
            
            tmp_placeholder2 = displaced_vertexes_underx[p+1]
            tmp_placeholder2.x = displaced_vertexes_underx[p+1].x/tmp_mag2
            tmp_placeholder2.y = displaced_vertexes_underx[p+1].y/tmp_mag2
            
            tmp_dot_product1 = dot_product(x_unit, tmp_placeholder1)
            tmp_dot_product2 = dot_product(x_unit, tmp_placeholder2)
            if tmp_dot_product1 > tmp_dot_product2:
                tmp_vertex1 = displaced_vertexes_underx[p]
                displaced_vertexes_underx[p] = displaced_vertexes_underx[p+1]
                displaced_vertexes_underx[p+1] = tmp_vertex1
 
    cdef size_t m
    cdef size_t n
    for m in range(vertexes_over_x_axis):
        free_vertexes[m] = displaced_vertexes_overx[m]
    

    for n in range(vertexes_over_x_axis, size):
        free_vertexes[n] = displaced_vertexes_underx[n-vertexes_over_x_axis]
        
 
    free(displaced_vertexes_overx)
    free(displaced_vertexes_underx)
#end function



#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* triangulate_vertices(Vertex* original_array, int varr_size) nogil:
    
    cdef Face* final_mesh = <Face*>malloc(sizeof(Face))
    cdef Face* bad_mesh = <Face*>malloc(sizeof(Face))
    cdef Vertex* free_vertices = <Vertex*>malloc(sizeof(Vertex))
    
    cdef int final_mesh_size = 0
    cdef int bad_mesh_size = 0
    cdef int free_vertices_size = 0

    cdef Face super_triangle = create_super_triangle(original_array, varr_size)
    
    final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, super_triangle)
    final_mesh_size += 1

    
    cdef int i
    cdef int j
    cdef int k
    cdef int m
    cdef int n
    cdef int l
    
    cdef Face tmp_face
    
    #main loop over all vertexes present in the og array
    for i in range(varr_size):
        #for each vertex loop over every Face in the final_mesh
        for j in range(final_mesh_size):
            #if the current vertex in the array is in the Face we are at, add the face to a bad_mesh array
            #increase size of bad mesh array by 1
            #deallocate the face from the final_mesh array
            #decrease the size of final mesh by 1
            if in_circumcircle(final_mesh[j], original_array[i]):
                bad_mesh = realloc_to_mesh(bad_mesh, bad_mesh_size, final_mesh[j])
                bad_mesh_size = bad_mesh_size + 1
                final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, j)
                final_mesh_size = final_mesh_size - 1
        
        #the second for iteration goes through the bad mesh and allocates all triangles vertixes to a free_array
        
        for m in range(bad_mesh_size):
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver1)
            free_vertices_size = free_vertices_size + 1
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver2)
            free_vertices_size = free_vertices_size + 1
            
            free_vertices = realloc_to_varray(free_vertices, free_vertices_size, bad_mesh[m].ver3)
            free_vertices_size = free_vertices_size + 1
        
        #in this loop the repeated vertixes are removed the remaining will be prepared for sorting by angle
        #the size of the array is kept track of everytime a vertex is remeaved due to it being repeated
        for k in range(free_vertices_size-1):
            for l in range(k+1, free_vertices_size):
                if free_vertices[l].vid_nr == free_vertices[k].vid_nr:
                    free_vertices = dealloc_from_varray(free_vertices, free_vertices_size, l)
                    free_vertices_size = free_vertices_size - 1
                    
        #now we sort the free_vertices by angle clockwise
        sort_by_angle_clockwise(free_vertices, original_array[i], free_vertices_size)
        
        #now we create a face in clockwise pairs for every 2-nearest couple in free_vertex + the origin, original)array[i]
        #then we allocate this face to final_mesh and update size before the main 'for' -i loop repeats.
        #first allocate the last Face to avoid 
        
        tmp_face = create_face(original_array[i], free_vertices[free_vertices_size-1], free_vertices[0], 0)
        final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
        final_mesh_size = final_mesh_size + 1
        for n in range(free_vertices_size-1):
            tmp_face = create_face(original_array[i], free_vertices[n], free_vertices[n+1], 0)
            final_mesh = realloc_to_mesh(final_mesh, final_mesh_size, tmp_face)
            final_mesh_size = final_mesh_size + 1
    free(bad_mesh)
    free(free_vertices)
    
    cdef int p
    cdef int q
    
    for p in range(final_mesh_size):
        if final_mesh[p].ver1.vid_nr < 0 or final_mesh[p].ver2.vid_nr < 0 or final_mesh[p].ver3.vid_nr < 0:
            final_mesh = dealloc_from_mesh(final_mesh, final_mesh_size, p)
            final_mesh_size = final_mesh_size - 1
    
    for q in range(final_mesh_size):
        final_mesh[q].fid_nr = q
    
    return final_mesh
            

    
    
    

"""unit test for angular sorting and using the dealloc functions on a mesh and varray"""
#start the array of Zero-Zero Vertices
#!! THIS ARRAY IS MALLOCED BUT NOT FREED BY DEFAULT !! THE SIZE HAS TO BE KEPT TRACK OF
cdef int varr_size = 7
cdef Vertex* vertex_array = alloc_varr(varr_size, 0)

vertex_array[0].x = 0.3
vertex_array[0].y = 2
vertex_array[1].x = 1.8
vertex_array[1].y = 0.1
vertex_array[2].x = -0.7
vertex_array[2].y = -0.1
vertex_array[3].x = 0.2
vertex_array[3].y = -2
vertex_array[4].x = -1.8
vertex_array[4].y = 6
vertex_array[5].x = -3.9
vertex_array[5].y = -2
vertex_array[6].x = 1.7
vertex_array[6].y = -3.4
cdef Vertex origin = create_vertex(0, 0, 20)


cdef Face* mesh = triangulate_vertices(vertex_array, varr_size)






free(vertex_array)









In [ ]:
#works
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void sort_by_angle_cclockwise(Vertex* free_vertexes, Vertex center_vertex, int size):
    cdef int vertexes_over_x_axis = 0
    cdef int vertexes_under_x_axis = 0
    """DON'T FORGET TO FREE"""
    ###
    cdef Vertex* displaced_vertexes_overx = <Vertex*>malloc(sizeof(Vertex))
    cdef Vertex* displaced_vertexes_underx = <Vertex*>malloc(sizeof(Vertex))
    ###
    

    cdef double x_newcoord
    cdef double y_newcoord
    
    cdef int i
    
    for i in range(size):
        x_newcoord = free_vertexes[i].x - center_vertex.x
        y_newcoord = free_vertexes[i].y - center_vertex.y

        if y_newcoord >= 0:
            displaced_vertexes_overx = realloc_to_varray(displaced_vertexes_overx, vertexes_over_x_axis, free_vertexes[i])
            vertexes_over_x_axis = vertexes_over_x_axis + 1
            
            
        elif y_newcoord < 0:
            displaced_vertexes_underx = realloc_to_varray(displaced_vertexes_underx, vertexes_under_x_axis, free_vertexes[i])
            vertexes_under_x_axis = vertexes_under_x_axis + 1
            
            

    cdef int j
    cdef int k
    cdef int l
    cdef int p
    cdef double tmp_mag1
    cdef double tmp_mag2
    cdef double tmp_dot_product1
    cdef double tmp_dot_product2
    cdef Vertex tmp_placeholder1
    cdef Vertex tmp_placeholder2
    cdef Vertex tmp_vertex1
    cdef double tmp_projection1
    cdef double tmp_projection2
    cdef Vertex x_unit = create_vertex(1, 0, 0)


    
    for j in range(vertexes_over_x_axis):
        for k in range(vertexes_over_x_axis-j-1):
            tmp_mag1 = (magnitude(displaced_vertexes_overx[k].x-center_vertex.x, displaced_vertexes_overx[k].y-center_vertex.y))
            tmp_mag2 = (magnitude(displaced_vertexes_overx[k+1].x-center_vertex.x, displaced_vertexes_overx[k+1].y-center_vertex.x))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            tmp_projection1 = (displaced_vertexes_overx[k].x-center_vertex.x)/tmp_mag1
            tmp_projection2 = (displaced_vertexes_overx[k+1].x-center_vertex.x)/tmp_mag2
            
            if tmp_projection1 <= tmp_projection2:
                tmp_vertex1 = displaced_vertexes_overx[k]
                displaced_vertexes_overx[k] = displaced_vertexes_overx[k+1]
                displaced_vertexes_overx[k+1] = tmp_vertex1

                
                
    for l in range(vertexes_under_x_axis):
        for p in range(vertexes_under_x_axis-l-1):
            tmp_mag1 = (magnitude(displaced_vertexes_underx[p].x-center_vertex.x, displaced_vertexes_underx[p].y-center_vertex.y))
            tmp_mag2 = (magnitude(displaced_vertexes_underx[p+1].x-center_vertex.x, displaced_vertexes_underx[p+1].y-center_vertex.y))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            tmp_projection1 = (displaced_vertexes_underx[k].x-center_vertex.x)/tmp_mag1
            tmp_projection2 = (displaced_vertexes_underx[k+1].x-center_vertex.x)/tmp_mag2

            
            if tmp_projection1 > tmp_projection2:
                tmp_vertex1 = displaced_vertexes_overx[k]
                displaced_vertexes_overx[k] = displaced_vertexes_overx[k+1]
                displaced_vertexes_overx[k+1] = tmp_vertex1
 
    cdef int m
    cdef int n
    for m in range(vertexes_over_x_axis):
        free_vertexes[m] = displaced_vertexes_overx[m]
    

    for n in range(vertexes_over_x_axis, size):
        free_vertexes[n] = displaced_vertexes_underx[n-vertexes_over_x_axis]
        
 
    free(displaced_vertexes_overx)
    free(displaced_vertexes_underx)

In [ ]:
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void meshify(Face* g_mesh, int gmsize, Face* b_mesh, int bmsize, Vertex current_vertex):
    
    for t in range(gmsize):
        print(g_mesh[t])
    
    cdef Face* new_g_mesh = <Face*>malloc(sizeof(Face))
    cdef Vertex* free_varray
    cdef int vsize = 0
    
    cdef int i
    cdef int j
    #fill thee free vertice array with all vertexes from the bad_mash array of faces
    for i in range(bmsize):
        free_varray = realloc_to_varray(free_varray, vsize, b_mesh[i].ver1)
        free_varray = realloc_to_varray(free_varray, vsize, b_mesh[i].ver2)
        free_varray = realloc_to_varray(free_varray, vsize, b_mesh[i].ver3)
        vsize = vsize + 3
    
    #empty the bad mesh array and reset size to 0 to prepare for next iteration out of this fx
    for j in range(bmsize):
        b_mesh = dealloc_from_mesh(b_mesh, bmsize, 0)
    bmsize = 0
        
    cdef int k = 0
    cdef int l = 0
    #dealloc all non-unique verrtices from the free arr mesh. use bubble
    #vsize = dealloc_non_uniques(free_varray, vsize)
    while k < vsize - 1:
        l = k + 1
        while l < vsize:
            if free_varray[l].vid_nr == free_varray[k].vid_nr:
                free_varray = dealloc_from_varray(free_varray, vsize, l)
                vsize = vsize - 1
            else:
                l = l + 1
        k = k + 1
    
                
    #sort thee free_v array by angle
    sort_by_angle_cclockwise(free_varray, current_vertex, vsize)
    
    #create a face for each 2 consecutive vertex pairs (v[i], v[i+1], current_vertex) and alloc to good mesh g_mesh
    cdef Face temp_face
    cdef int m
    for m in range(vsize):
        temp_face = create_face(current_vertex, free_varray[m], free_varray[(m+1)%vsize], 0)
        g_mesh = realloc_to_mesh(g_mesh, gmsize, temp_face)
        gmsize = gmsize + 1
        
    #empty free vertex arr for next iteration of current vertex in main loop
    free(free_varray)